<a href="https://colab.research.google.com/github/annikarichardson2024/ChatBot/blob/main/ChatbotEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np


In [3]:
questions = ['Hello', 'How are you?', 'What is your name?', 'What is TensorFlow?', 'Do you like Python?']
answers = ['Hello there!', 'I am fine.', 'I am a bot.', 'TensorFlow is a machine learning library.', 'Yes, Python is great!']

In [4]:
# Tokenize sentences
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_questions = tokenizer.texts_to_sequences(questions)
sequences_answers = tokenizer.texts_to_sequences(answers)

In [9]:
max_length = max(max(len(seq) for seq in sequences_questions), max(len(seq) for seq in sequences_answers))
padded_questions = tf.keras.preprocessing.sequence.pad_sequences(sequences_questions, maxlen=max_length, padding='post')
padded_answers = tf.keras.preprocessing.sequence.pad_sequences(sequences_answers, maxlen=max_length, padding='post')

# voab size
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# model parameters
embedding_dim = 200
units = 1000

# Define the encoder model
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder model
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the seq2seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [11]:
# decoder input data that just contains the start token and the rest is all 0
# It's a simplified approach, in practice, you would use teacher forcing or another technique
decoder_input_data = np.zeros_like(padded_answers)
decoder_input_data[:, 0] = 1  # Assuming 1 is the start token

# Train the model
model.fit([padded_questions, decoder_input_data], np.expand_dims(padded_answers, -1), batch_size=2, epochs=100)


Epoch 1/100
2/2 [==============================] - 4s 128ms/step - loss: 3.0506 - accuracy: 0.1667
Epoch 2/100
2/2 [==============================] - 0s 15ms/step - loss: 2.8484 - accuracy: 0.2917
Epoch 3/100
2/2 [==============================] - 0s 16ms/step - loss: 2.4829 - accuracy: 0.2917
Epoch 4/100
2/2 [==============================] - 0s 15ms/step - loss: 2.8031 - accuracy: 0.3333
Epoch 5/100
2/2 [==============================] - 0s 15ms/step - loss: 2.3458 - accuracy: 0.3750
Epoch 6/100
2/2 [==============================] - 0s 15ms/step - loss: 2.3195 - accuracy: 0.3750
Epoch 7/100
2/2 [==============================] - 0s 14ms/step - loss: 2.1941 - accuracy: 0.4167
Epoch 8/100
2/2 [==============================] - 0s 118ms/step - loss: 2.1252 - accuracy: 0.4167
Epoch 9/100
2/2 [==============================] - 0s 16ms/step - loss: 2.1732 - accuracy: 0.3333
Epoch 10/100
2/2 [==============================] - 0s 15ms/step - loss: 2.0323 - accuracy: 0.3750
Epoch 11/100
2/2 

In [7]:
# Step 5: Implement the chat function
def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1  # start token
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:  # end token
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

def chat_with_bot(input_text):
    input_sequence = preprocess_input_text(input_text)
    response_sequence = generate_response(input_sequence)
    response_text = sequence_to_text(response_sequence[0])
    return response_text

# Step 6: Chat with the bot
input_text = "What is TensorFlow?"
print(f"You: {input_text}")
print(f"Bot: {chat_with_bot(input_text)}")

You: What is TensorFlow?
1/1 [==============================] - 0s 21ms/step
Bot: tensorflow a machine learning


In [ ]:
# Interactive chat
print("Start chatting with the bot! Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = chat_with_bot(user_input)
    print(f"Bot: {response}")


Start chatting with the bot! Type 'quit' to exit.
